In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

import statsmodels.api as sm
from statsmodels.formula.api import ols

from tsfresh.feature_selection import relevance as feature_relevance
from tsfresh import extract_features, select_features

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import gc
import sys

#Utilities
import warnings
from tqdm import tqdm
import time
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x) # floats view format

## Globals

In [2]:
data_folder = '/home/vlad/DiplomaWork/data'
RS = 42
# categories = ['02-11', '02-12', '02-13', '02-14']

## Utilities

In [3]:
# Get execution time and data size
def execution_time(func):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        res = func(*args, **kwargs)
        t2 = time.time()
        ex_mins = (t2 - t1)//60
        ex_sec =  (t2 - t1)%60
        print(f'Process time: {ex_mins} min, {int(ex_sec)} sec')
        if res is not None:
            print(f'Data size: {res.shape[0]} x {res.shape[1]}')
            return res
    return wrapper

# Check NaN ratio per column
def get_nan_ratio(df):
    length = df.shape[0]
    ratios = []
    for col in df.columns:
        nan_ratio = 100 * (1 - df[col].count() / length)
        ratios.append(nan_ratio)
    stat_df = pd.DataFrame()
    stat_df['Column'] = df.columns
    stat_df['NaN_ratio_prc'] = ratios
    return stat_df.sort_values(by = ['NaN_ratio_prc'], ascending = False)

## Data read

In [4]:
df = pd.read_csv(f'{data_folder}/PRE_ABT_POS.csv')
products = pd.read_csv(f'{data_folder}/PRODUCTS.csv')
# products_ids = products[products['PRODUCT_LVL_CD2'].isin(categories)]['PRODUCT_ID'].values
products_ids = products['PRODUCT_ID'].values
df = df[df['product_id'].isin(products_ids)]
df = df.rename(columns = {'product_id': 'PRODUCT_ID'})

print(df.shape)
df.head()

(17394831, 24)


,PRODUCT_ID,LOCATION_id,customer_id,distr_channel_id,period_dt,PRICE_REG,PRICE_ACT,PRICE_PROMO,PRICE_DISCOUNT_RATE,PROMO_1001,PROMO_1002,PROMO_1003,PROMO_FLG,PROMO_FLG_DR,DEFICIT_FLG1,DEFICIT_FLG2,STOCK_QTY,TGT_QTY,TGT_QTY_R,NUM_AUTHORIZATION,out_part,ml_part,vf_part,bl_part
0,203945,4401,-6,-2,22514,81.0000,81.0000,81.0000,0.0000,0,0,0,0,0.0000,0,0,NaN,NaN,NaN,1,NaN,13,16,2
1,203945,4401,-6,-2,22563,83.0000,83.0000,83.0000,0.0000,0,0,0,0,0.0000,0,0,NaN,NaN,NaN,1,NaN,13,16,2
2,203945,4401,-6,-2,22576,83.0000,83.0000,83.0000,0.0000,0,0,0,0,0.0000,0,0,NaN,NaN,NaN,1,NaN,13,16,2
3,203945,9502,-6,-2,22542,NaN,NaN,NaN,0.0000,0,0,0,0,0.0000,0,0,NaN,NaN,NaN,1,NaN,5,42,2
4,203945,9502,-6,-2,22555,NaN,NaN,NaN,0.0000,0,0,0,0,0.0000,0,0,NaN,NaN,NaN,1,NaN,5,42,2


In [6]:
len(df[['PRODUCT_ID', 'LOCATION_id']].drop_duplicates())

22015

## Preprocessing

In [5]:
def preprocess_date(df):
    df['POS_DT'] = pd.to_datetime(df['POS_DT'], format="%d%b%Y")
    df['MONTH'] = df['POS_DT'].dt.month
    df['YEAR'] = df['POS_DT'].dt.year
    return df.drop(columns = ['POS_DT'])

def add_days_to_date(date, days):
    added_date = pd.to_datetime(date) + timedelta(days=days)
    added_date = added_date.strftime("%Y-%m-%d")
    return added_date

def dates_giver(df):
    df['DATE'] = '01-01-1960'
    df['DATE'] = pd.to_datetime(df['DATE'], format="%d-%m-%Y")
    df['DATE'] = df.apply(lambda x: add_days_to_date(x['DATE'], x['period_dt']), axis = 1)
    data = df[['DATE', 'PRODUCT_ID', 'LOCATION_id', 'PRICE_REG', 'PRICE_PROMO', 'STOCK_QTY', 'TGT_QTY']]
    data['week_number'] = pd.to_datetime(data['DATE'], format = "%Y-%m-%d").dt.week
    data['month_number'] = pd.to_datetime(data['DATE'], format = "%Y-%m-%d").dt.month
    data['year'] = pd.to_datetime(data['DATE'], format = "%Y-%m-%d").dt.year
    data['day'] = pd.to_datetime(data['DATE'], format = "%Y-%m-%d").dt.day
    return data

@execution_time
def preprocessing(df):
    df = dates_giver(df)
    # data = data.drop(columns = ['DATE'])
    df = df[df['TGT_QTY'].notnull()]  
    df = df[df['STOCK_QTY'].notnull()]
    df = df.reset_index(drop = True)
    return df
    

In [6]:
df = preprocessing(df)
df.head()

Process time: 5.0 min, 2 sec
Data size: 13020968 x 11


,DATE,PRODUCT_ID,LOCATION_id,PRICE_REG,PRICE_PROMO,STOCK_QTY,TGT_QTY,week_number,month_number,year,day
0,2019-02-22,205880,145,105.0000,94.5000,2.0000,0.0000,8,2,2019,22
1,2019-03-16,205880,145,105.0000,89.4700,2.0000,0.0000,11,3,2019,16
2,2019-05-13,205880,145,105.0000,94.5000,2.0000,0.0000,20,5,2019,13
3,2019-07-28,205880,145,105.0000,105.0000,2.0000,0.0000,30,7,2019,28
4,2019-09-19,205880,145,105.0000,105.0000,2.0000,0.0000,38,9,2019,19


In [7]:
df['DATE'].min()

'2018-01-01'

In [8]:
df['DATE'].max()

'2021-08-15'

## Feature creation

In [9]:
def add_holidays(df):
    df['Feb23'] = ((df['month_number'] == 2) & (df['day'] == 23)).astype(int)
    df['May1'] = ((df['month_number'] == 5) & (df['day'] == 1)).astype(int)
    df['Jun12'] = ((df['month_number'] == 6) & (df['day'] == 12)).astype(int)
    df['Nov4'] = ((df['month_number'] == 11) & (df['day'] == 4)).astype(int)
    df['NY'] = ((df['month_number'] == 12) & (df['day'].isin([24, 25, 26, 27, 28, 29, 30, 31]))).astype(int)
    return df

def get_weekly_sales(df):
    df_sums = df.groupby(by = ['PRODUCT_ID', 'LOCATION_id', 'year','week_number']).sum().reset_index()
    df_sums = df_sums[['PRODUCT_ID', 'LOCATION_id', 'year','week_number', 'TGT_QTY']]
    return df_sums

def get_week_mean_price(df):
    df_means = df.groupby(by = ['PRODUCT_ID', 'LOCATION_id', 'year','week_number']).mean().reset_index()
    df_means = df_means[['PRODUCT_ID', 'LOCATION_id', 'year','week_number', 'PRICE_REG', 'PRICE_PROMO']]
    return df_means
    
def get_weekly_stocks(df):
    weekly_stocks = df.sort_values(by = ['PRODUCT_ID', 'LOCATION_id', 'DATE'])
    weekly_stocks = weekly_stocks[~weekly_stocks['STOCK_QTY'].isnull()]
    weekly_stocks = weekly_stocks.drop_duplicates(subset = ['PRODUCT_ID', 'LOCATION_id', 'week_number','year'], keep = 'first')
    weekly_stocks = weekly_stocks[['PRODUCT_ID', 'LOCATION_id', 'week_number','year', 'STOCK_QTY']]
    return weekly_stocks
    
def get_ohe_holidays(df):
    holi_cols = ['Feb23','May1','Jun12', 'Nov4', 'NY']
    df = add_holidays(df)
    holiweeks = df.groupby(by = ['year', 'week_number']).sum()
    holiweeks = holiweeks[holi_cols]
    for col in holi_cols:
        holiweeks[col] = (holiweeks[col] > 0).astype(int)
    holiweeks = holiweeks.reset_index()
    return holiweeks

def top_3_min_prices(df_means):
    agr_cols = ['LOCATION_id', 'year', 'week_number']
    to_agr_cols = ['PRICE_REG']
    grouped = df_means[agr_cols + to_agr_cols].groupby(by = agr_cols)['PRICE_REG']
    df_means['1st_min_price'] = grouped.transform(lambda x: x.nsmallest(1).min())
    df_means['2nd_min_price'] = grouped.transform(lambda x: x.nsmallest(2).min())
    df_means['3rd_min_price'] = grouped.transform(lambda x: x.nsmallest(3).min())
    return df_means
    
@execution_time
def agregate_data(df):
    df_sums = get_weekly_sales(df) # Total sales per week
    df_means = get_week_mean_price(df) # Mean price per week
    weekly_stocks = get_weekly_stocks(df) # Stock per week
    holiweeks = get_ohe_holidays(df) # Get OHE holidays
    df_means = top_3_min_prices(df_means) # Get 3 min prices of concurents per week 
    
    # Combining agr dfs together
    ped_df = df_sums.merge(df_means, how = 'left', on = ['PRODUCT_ID', 'LOCATION_id', 'year','week_number'])
    ped_df = ped_df.merge(holiweeks, how = 'left', on = ['year','week_number'])
    
    ped_df['PRICE_PROMO'] = ped_df['PRICE_PROMO'].fillna(ped_df['PRICE_REG'])
    ped_df['promo_discount_%'] = (1 - ped_df['PRICE_PROMO'] / ped_df['PRICE_REG']) * 100 # Discount Calculation
    return ped_df
    

In [10]:
ped_df = agregate_data(df)

ped_df = ped_df.dropna().reset_index(drop = True)
print('Non Nan Data size:', ped_df.shape)
ped_df.head()

Process time: 0.0 min, 38 sec
Data size: 1883081 x 16
Non Nan Data size: (1440304, 16)


,PRODUCT_ID,LOCATION_id,year,week_number,TGT_QTY,PRICE_REG,PRICE_PROMO,1st_min_price,2nd_min_price,3rd_min_price,Feb23,May1,Jun12,Nov4,NY,promo_discount_%
0,370,223,2018,1,0.0000,166.2500,154.9333,0.9913,0.9913,0.9913,0,0,0,0,1,6.8070
1,370,223,2018,2,0.0000,166.0000,166.0000,1.0400,1.0400,1.0400,0,0,0,0,0,0.0000
2,370,223,2018,3,1.0000,166.0000,149.4000,1.0400,1.0400,1.0400,0,0,0,0,0,10.0000
3,370,223,2018,4,0.0000,166.0000,149.4000,1.0400,1.0400,1.0400,0,0,0,0,0,10.0000
4,370,223,2018,5,0.0000,166.0000,149.4000,0.7600,0.7600,0.7600,0,0,0,0,0,10.0000


## Pairs selection
- (Shop - product) pairs for experiments

In [11]:
def get_pairs_for_experiment(ped_df, quantile = 0.5):
    popular_pairs = ped_df.groupby(['PRODUCT_ID', 'LOCATION_id']).size().reset_index(name='counts')
    popular_pairs = popular_pairs.sort_values(by='counts', ascending=False).reset_index(drop=True)
    print('Pairs with count >', popular_pairs['counts'].quantile(quantile), 'selected')
    popular_pairs = popular_pairs[popular_pairs['counts'] > popular_pairs['counts'].quantile(quantile)]
    pairs_for_experiment = list(popular_pairs[['PRODUCT_ID', 'LOCATION_id']].to_records(index=False))
    popular_pairs = popular_pairs.rename(columns = {'PRODUCT_ID' : 'SKU', 'LOCATION_id' : 'Store'})
    print(f'Total pairs selected: {len(pairs_for_experiment)}')
    return popular_pairs, pairs_for_experiment

In [12]:
popular_pairs_df, pairs_for_experiment = get_pairs_for_experiment(ped_df, quantile = 0.5)

Pairs with count > 57.0 selected
Total pairs selected: 8962


In [13]:
ped_df

,PRODUCT_ID,LOCATION_id,year,week_number,TGT_QTY,PRICE_REG,PRICE_PROMO,1st_min_price,2nd_min_price,3rd_min_price,Feb23,May1,Jun12,Nov4,NY,promo_discount_%
0,370,223,2018,1,0.0000,166.2500,154.9333,0.9913,0.9913,0.9913,0,0,0,0,1,6.8070
1,370,223,2018,2,0.0000,166.0000,166.0000,1.0400,1.0400,1.0400,0,0,0,0,0,0.0000
2,370,223,2018,3,1.0000,166.0000,149.4000,1.0400,1.0400,1.0400,0,0,0,0,0,10.0000
3,370,223,2018,4,0.0000,166.0000,149.4000,1.0400,1.0400,1.0400,0,0,0,0,0,10.0000
4,370,223,2018,5,0.0000,166.0000,149.4000,0.7600,0.7600,0.7600,0,0,0,0,0,10.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440299,5201142,7002,2021,32,0.0000,830.0000,830.0000,1.5400,1.5400,1.5400,0,0,0,0,0,0.0000
1440300,5300493,51,2021,32,1.0000,83.0000,83.0000,8.0000,8.0000,8.0000,0,0,0,0,0,0.0000
1440301,5500002,153,2021,31,1.0000,170.0000,170.0000,1.0800,1.0800,1.0800,0,0,0,0,0,0.0000
1440302,5500002,153,2021,32,0.0000,170.0000,170.0000,1.0800,1.0800,1.0800,0,0,0,0,0,0.0000


## Log-Log regression

In [14]:
def transform(x, ttype):
    '''
    Accepts vector and performs transformation on it based on ttype
    '''
    if ttype == 'log':
        return np.log1p(x)
    
    return np.array(x)

def coef(model):
    """
    return regression coefficient of model
    """
    return model.params[1] if len(model.params) > 1 else model.params[0]

def pvalue(model):
    """
    return regression coefficient's t-test p-value of model
    """
    return model.pvalues[1] if len(model.pvalues) > 1 else model.pvalues[0]

def elasticity(X, y, model, model_type):
    '''
    Computes elasticity of model based on X, y, model itself and model's type
    '''
    if model_type == 'lin-lin':
        return coef(model) * np.mean(X) / np.mean(y)
    elif model_type == 'log-lin':
        return coef(model) * np.mean(X)
    elif model_type == 'log-log':
        return coef(model)
    
def PEDmodel(X, y, model_type='lin-lin'):
    '''
    Makes PED model based on input data X, y and model type
    '''
    _types = model_type.split('-')
    
    act_x = transform(X, _types[1])
    act_x = sm.add_constant(act_x)
    
    act_y = transform(y, _types[0])
    return sm.OLS(act_y, act_x).fit()

def get_model_stats(X, y, model, model_type):
    '''
    returns model's statistics
        regression coefficient,
        R-squared,
        T-test p-value,
        elasticity
    '''
    elas = elasticity(X, y, model, model_type)
    return coef(model), model.rsquared, pvalue(model), elas

def GeneratePEDModels(dataset, model_type):
    col_list = ['PRICE_REG', 'Feb23','May1','Jun12','Nov4','NY']
    sku_id, store_id, elas, qty, ttpvalue = [], [], [], [], []
    iter_count = 0
    for good_id, shop_id in tqdm(pairs_for_experiment):
        data = dataset[(dataset['LOCATION_id'] == shop_id) & (dataset['PRODUCT_ID'] == good_id)]
        try: # TODO (Understand error)
            model = PEDmodel(data[col_list], data['TGT_QTY'], model_type)
            c, r2, tp, e = get_model_stats(data[col_list], data['TGT_QTY'], model, model_type)
            sku_id.append(good_id)
            store_id.append(shop_id)
            elas.append(e)
            ttpvalue.append(tp)
            qty.append(data['TGT_QTY'].sum() / data.shape[0])
        except:
            continue
    
    return pd.DataFrame({
        'SKU': sku_id,
        'Store': store_id,
        'Elasticity': elas,
        'Qty': qty,
        'P_value': ttpvalue
    })

def get_robust_pairs(elast_df):
    ok_elasts = len(elast_df[(elast_df['Elasticity'] > -10) & (elast_df['Elasticity'] < 0)])
    ok_pvals = len(elast_df['P_value'] < 0.05)
    robust_elasts = elast_df[(elast_df['Elasticity'] > -10) & 
                             (elast_df['Elasticity'] < 0) & 
                             (elast_df['P_value'] < 0.05)]
    
    print('# Normal elasticities:', ok_elasts)
    print('# Normal p-values:', ok_pvals)
    print('# Robust elasticities:', len(robust_elasts))
    print('% Robust elasticities:', np.round((len(robust_elasts) / len(elast_df)) * 100, 2))
    return robust_elasts

In [38]:
def ret_head(df, ratio):
    partition = int(np.round(len(df) * ratio))
    return df.head(partition)

def ret_tail(df, ratio):
    partition = int(np.round(len(df) * ratio))
    return df.tail(partition)

def make_crops(ped_df, crop_type = 'backward'):
    if crop_type not in ['backward', 'forward']:
        print('crop_type is not correct')
        return None

    croped_frames = []
    group_cols = ['PRODUCT_ID', 'LOCATION_id']
    for i in tqdm(np.arange(0.1, 0.6, 0.1)):
        if crop_type == 'backward':
            croped_ped_df = ped_df.groupby(by = group_cols).apply(lambda x: ret_head(x, 1-i))
            croped_ped_df = croped_ped_df.drop(columns = group_cols).reset_index().drop(columns = ['level_2'])
        else:
            croped_ped_df = ped_df.groupby(by = group_cols).apply(lambda x: ret_tail(x, 1-i))
            croped_ped_df = croped_ped_df.drop(columns = group_cols).reset_index().drop(columns = ['level_2'])

        croped_frames.append(croped_ped_df)
    return croped_frames

def price_plot_prep(ped_df):
    plot_df = ped_df[['PRODUCT_ID', 'LOCATION_id', 'year', 'week_number', 'PRICE_REG', 'TGT_QTY']]
    plot_df['week'] = plot_df['year'].astype('str')+'_'+plot_df['week_number'].astype('str')
    plot_df['pair'] = list(zip(plot_df['PRODUCT_ID'], plot_df['LOCATION_id']))
    plot_df['raw_pair'] = plot_df['PRODUCT_ID'].astype(str) + '_' + plot_df['LOCATION_id'].astype(str)    
    cat_dict, cnt = dict(), 1
    for pair in plot_df['raw_pair'].unique():
        cat_dict[pair] = cnt
        cnt+=1
    
    keys, vals = cat_dict.keys(), cat_dict.values()
    replacer = pd.DataFrame(data = {'raw_pair' : keys, 'id' : vals})
    plot_df = plot_df.merge(replacer, how = 'left', on = ['raw_pair'])
#     plot_df['id'] = plot_df['raw_pair'].replace(cat_dict) # TODO speedup with merge
    return plot_df[['id', 'pair','raw_pair', 'PRODUCT_ID', 'LOCATION_id', 'week', 'PRICE_REG', 'TGT_QTY']]

def get_ts_features(plot_df):
    df_features = extract_features(plot_df[['id', 'week', 'PRICE_REG', 'TGT_QTY']], column_id="id", column_sort="week")
    df_features = df_features.reset_index().rename(columns = {'index': 'id'})
    df_features = df_features.merge(plot_df[['id', 'PRODUCT_ID', 'LOCATION_id']].drop_duplicates(), how = 'left', on = ['id'])
    df_features = df_features.rename(columns = {'PRODUCT_ID' : 'SKU', 'LOCATION_id': 'Store'})
    return df_features

def clf_prep(croped_frame, ts_features = True):
    elast_df = GeneratePEDModels(croped_frame, model_type = 'log-log')
    counts = croped_frame.groupby(by = ['PRODUCT_ID', 'LOCATION_id']).count()[['year']].reset_index()
    
    rename_dict = {'PRODUCT_ID' : 'SKU', 'LOCATION_id': 'Store', 'year' : 'count'}
    elast_df = elast_df.merge(counts.rename(columns = rename_dict), how = 'left', on = ['SKU', 'Store'])
    if ts_features:
        plot_df = price_plot_prep(croped_frame)
        elast_df = elast_df.merge(get_ts_features(plot_df), how = 'left', on = ['SKU', 'Store'])
    return elast_df


        

In [39]:
columns = ['SKU', 'Store', 'Elasticity', 'Qty', 'P_value', 'count', '__type__',
           'id', 'PRICE_REG__index_mass_quantile__q_0.1',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_2',
           'TGT_QTY__ratio_value_number_to_time_series_length',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_15',
           'PRICE_REG__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_27',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_37',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_28',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_25',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_13',
           'PRICE_REG__index_mass_quantile__q_0.2',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_5',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_8',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_39',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_35',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_30',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_4',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',
           'PRICE_REG__variation_coefficient',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_11',
           'TGT_QTY__percentage_of_reoccurring_datapoints_to_all_datapoints',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_16',
           'PRICE_REG__linear_trend__attr_"stderr"',
           'PRICE_REG__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"max"',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_31',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_14',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_26',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_7',
           'PRICE_REG__fft_aggregated__aggtype_"skew"',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_20',
           'PRICE_REG__agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"max"',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_34',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_33',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_38',
           'PRICE_REG__agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"var"',
           'TGT_QTY__fft_coefficient__attr_"abs"__coeff_10',
           'PRICE_REG__spkt_welch_density__coeff_8',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_3',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_4',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_36',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_10',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_12',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_18',
           'PRICE_REG__fft_aggregated__aggtype_"centroid"',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
           'PRICE_REG__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',
           'TGT_QTY__lempel_ziv_complexity__bins_100',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_27',
           'TGT_QTY__variance_larger_than_standard_deviation',
           'PRICE_REG__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
           'PRICE_REG__mean_n_absolute_max__number_of_maxima_7',
           'PRICE_REG__spkt_welch_density__coeff_5',
           'PRICE_REG__fft_coefficient__attr_"abs"__coeff_6',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',
           'TGT_QTY__index_mass_quantile__q_0.2',
           'PRICE_REG__agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"',
           'PRICE_REG__sum_of_reoccurring_values',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_9',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_13',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',
           'TGT_QTY__sum_of_reoccurring_values',
           'TGT_QTY__permutation_entropy__dimension_3__tau_1',
           'TGT_QTY__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
           'TGT_QTY__range_count__max_1__min_-1',
           'TGT_QTY__symmetry_looking__r_0.05',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_25',
           'PRICE_REG__agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"min"',
           'PRICE_REG__fft_coefficient__attr_"real"__coeff_14',
           'PRICE_REG__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',
           'PRICE_REG__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
           'PRICE_REG__fft_coefficient__attr_"imag"__coeff_6',
           'TGT_QTY__count_below_mean', 'TGT_QTY__index_mass_quantile__q_0.3']

b_croped_frames = make_crops(ped_df, crop_type = 'backward')
f_croped_frames = make_crops(ped_df, crop_type = 'forward')


b_elasts, f_elasts = [], []
for i in range(len(b_croped_frames)):
    bc = clf_prep(b_croped_frames[i])
    bc['__type__'] = f'b{int((i+1)*10)}%'
    
    fc = clf_prep(f_croped_frames[i])
    fc['__type__'] = f'f{int((i+1)*10)}%'
    
    b_elasts.append(bc[columns])
    f_elasts.append(fc[columns])
    
    
# Twice croped extention
bb_croped_frames, ff_croped_frames = [], []
fb_croped_frames, bf_croped_frames = [], []

print('Collecting type2 b_croped')
for bcr in b_croped_frames:
    bb_croped_frames.append(make_crops(bcr, crop_type = 'backward'))
    fb_croped_frames.append(make_crops(bcr, crop_type = 'forward'))
    
print('Collecting type2 f_croped')
for fcr in f_croped_frames:
    bf_croped_frames.append(make_crops(fcr, crop_type = 'backward'))
    ff_croped_frames.append(make_crops(fcr, crop_type = 'forward'))
    
    
bb_elasts, fb_elasts, ff_elasts, bf_elasts = [],[],[],[]
print('clf_prep for bb_croped')
for i in range(len(bb_croped_frames)):
    for j in range(len(bb_croped_frames[i])):
        bc = clf_prep(bb_croped_frames[i][j], ts_features = False)
        bc['__type__'] = f'b{int((i+1)*10)}%_b{int((j+1)*10)}%'
        
        fc = clf_prep(fb_croped_frames[i][j], ts_features = False)
        fc['__type__'] = f'b{int((i+1)*10)}%_f{int((j+1)*10)}%'
        
        bb_elasts.append(bc)
        fb_elasts.append(fc)

print('clf_prep for ff_croped')
for i in range(len(ff_croped_frames)):
    for j in range(len(ff_croped_frames[i])):
        fc = clf_prep(ff_croped_frames[i][j], ts_features = False)
        fc['__type__'] = f'f{int((i+1)*10)}%_f{int((j+1)*10)}%'
        
        bc = clf_prep(bf_croped_frames[i][j], ts_features = False)
        bc['__type__'] = f'f{int((i+1)*10)}%_b{int((j+1)*10)}%'
        
        ff_elasts.append(fc)
        bf_elasts.append(bc)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 15/8962 [00:00<01:02, 143.25it/s]

clf_prep for bb_croped


  0%|          | 15/8962 [00:00<01:01, 144.81it/s]

clf_prep for ff_croped


100%|██████████| 8962/8962 [00:40<00:00, 219.50it/s]


In [40]:
double_crops = pd.DataFrame()
for i in range(len(ff_elasts)):
    double_crops = double_crops.append(ff_elasts[i])
    double_crops = double_crops.append(fb_elasts[i])
    double_crops = double_crops.append(bb_elasts[i])
    double_crops = double_crops.append(bf_elasts[i])

double_crops['__type1__'] = double_crops['__type__'].str[:4]
double_crops['__type2__'] = double_crops['__type__'].str[-4:]
double_crops.head()

,SKU,Store,Elasticity,Qty,P_value,count,__type__,__type1__,__type2__
0,370,223,0.0000,0.0000,NaN,154,f10%_f10%,f10%,f10%
1,181916,29,-0.5750,0.3766,0.0568,154,f10%_f10%,f10%,f10%
2,51786,148,0.0399,0.1429,0.8470,154,f10%_f10%,f10%,f10%
3,181621,22,-0.5046,0.4610,0.1984,154,f10%_f10%,f10%,f10%
4,51893,128,0.5100,0.5260,0.1755,154,f10%_f10%,f10%,f10%


## Classification frame creation

In [41]:
clf_df = clf_prep(ped_df)
clf_df['__type__'] = f'n0%'
clf_df = clf_df[columns]
for i in range(len(b_elasts)):
    clf_df = clf_df.append(b_elasts[i])
    clf_df = clf_df.append(f_elasts[i])
    
b_elasts, f_elasts = [],[]
clf_df = clf_df.reset_index(drop = True)

print('Frame shape:', clf_df.shape)
clf_df.head()

Feature Extraction: 100%|██████████| 70/70 [02:11<00:00,  1.87s/it]


Frame shape: (98582, 85)


,SKU,Store,Elasticity,Qty,P_value,count,__type__,id,PRICE_REG__index_mass_quantile__q_0.1,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_2",TGT_QTY__ratio_value_number_to_time_series_length,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_15","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_27","PRICE_REG__fft_coefficient__attr_""abs""__coeff_37","PRICE_REG__fft_coefficient__attr_""abs""__coeff_28","PRICE_REG__fft_coefficient__attr_""abs""__coeff_25","PRICE_REG__fft_coefficient__attr_""abs""__coeff_13",PRICE_REG__index_mass_quantile__q_0.2,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_5","PRICE_REG__fft_coefficient__attr_""abs""__coeff_8","PRICE_REG__fft_coefficient__attr_""abs""__coeff_39","PRICE_REG__fft_coefficient__attr_""abs""__coeff_35","PRICE_REG__fft_coefficient__attr_""abs""__coeff_30","PRICE_REG__fft_coefficient__attr_""abs""__coeff_4","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0",PRICE_REG__variation_coefficient,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_11",TGT_QTY__percentage_of_reoccurring_datapoints_to_all_datapoints,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_16","PRICE_REG__linear_trend__attr_""stderr""","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_31","PRICE_REG__fft_coefficient__attr_""abs""__coeff_14","PRICE_REG__fft_coefficient__attr_""abs""__coeff_26","PRICE_REG__fft_coefficient__attr_""abs""__coeff_7","PRICE_REG__fft_aggregated__aggtype_""skew""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","PRICE_REG__fft_coefficient__attr_""abs""__coeff_20","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_34","PRICE_REG__fft_coefficient__attr_""abs""__coeff_33","PRICE_REG__fft_coefficient__attr_""abs""__coeff_38","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","TGT_QTY__fft_coefficient__attr_""abs""__coeff_10",PRICE_REG__spkt_welch_density__coeff_8,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_3","PRICE_REG__fft_coefficient__attr_""imag""__coeff_4","PRICE_REG__fft_coefficient__attr_""abs""__coeff_36","PRICE_REG__fft_coefficient__attr_""abs""__coeff_10","PRICE_REG__fft_coefficient__attr_""abs""__coeff_12","PRICE_REG__fft_coefficient__attr_""abs""__coeff_18","PRICE_REG__fft_aggregated__aggtype_""centroid""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","PRICE_REG__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__lempel_ziv_complexity__bins_100,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_27",TGT_QTY__variance_larger_than_standard_deviation,PRICE_REG__energy_ratio_by_chunks__num_segments_10__segment_focus_1,PRICE_REG__mean_n_absolute_max__number_of_maxima_7,PRICE_REG__spkt_welch_density__coeff_5,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_6","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2",TGT_QTY__index_mass_quantile__q_0.2,"PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""min""",PRICE_REG__sum_of_reoccurring_values,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_9","PRICE_REG__fft_coefficient__attr_""imag""__coeff_13","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__sum_of_reoccurring_values,TGT_QTY__permutation_entropy__dimension_3__tau_1,TGT_QTY__energy_ratio_by_chunks__num_segments_10__segment_focus_0,TGT_QTY__range_count__max_1__min_-1,TGT_QTY__symmetry_looking__r_0.05,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_25","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""","PRICE_REG__fft_coefficient__attr_""

## Product Features

In [42]:
def ohe_country(df, postfix):
    df['PRODUCT_ATTR_VL'] = df['PRODUCT_ATTR_VL'] + '_' + postfix
    top_5 = df.groupby(by = ['PRODUCT_ATTR_VL']).count().sort_values(by = ['PRODUCT_ID']).tail(5).reset_index()
    top_5 = top_5['PRODUCT_ATTR_VL'].to_list()
    one_hot = pd.get_dummies(df['PRODUCT_ATTR_VL'])
    df = df.drop('PRODUCT_ATTR_VL',axis = 1)
    df = df.join(one_hot)
    df[f'world_{postfix}'] = 0
    df = df.reset_index(drop = True)
    for col in df.columns:
        if (col not in ['PRODUCT_ID', f'world_{postfix}']) and (col not in top_5):
            df[f'world_{postfix}'] += df[col]
    df = df[['PRODUCT_ID', f'world_{postfix}'] + top_5]
    return df

def preprocess_weight(value):
    if type(value) is str:
        value = value.replace(',','.')
    try:
        return float(value)
    except:
        return np.nan
    
def guarantee_prep(value):
    try:
        if type(value) is not str:
            return 12
        sp = value.split()
        if len(sp) > 2:
            return 12
        coef = 1
        if ('ГОД') in sp[1]:
            coef = 12
        sp[0] = float(sp[0].replace(',', '.'))
        return sp[0] * coef
    except:
        return 12


prod_feat = pd.read_csv(f'{data_folder}/PRODUCT_ATTR _.csv')    
weight = prod_feat[prod_feat['PRODUCT_ATTR_NM'] == 'Вес'][['PRODUCT_ID', 'PRODUCT_ATTR_VL']].drop_duplicates()
guarantee = prod_feat[prod_feat['PRODUCT_ATTR_NM'] == 'Гарантия'][['PRODUCT_ID', 'PRODUCT_ATTR_VL']].drop_duplicates()
brand_country = prod_feat[prod_feat['PRODUCT_ATTR_NM'] == 'Страна бренда'][['PRODUCT_ID', 'PRODUCT_ATTR_VL']].drop_duplicates()
fab_country = prod_feat[prod_feat['PRODUCT_ATTR_NM'] == 'Страна производитель'][['PRODUCT_ID', 'PRODUCT_ATTR_VL']].drop_duplicates()

brand_country = ohe_country(brand_country, 'brand')
fab_country = ohe_country(fab_country, 'manuf')
weight['PRODUCT_ATTR_VL'] = weight.apply(lambda x: preprocess_weight(x['PRODUCT_ATTR_VL']), axis = 1)
weight = weight.rename(columns = {'PRODUCT_ATTR_VL' : 'weight'})
guarantee['PRODUCT_ATTR_VL'] = guarantee.apply(lambda x: guarantee_prep(x['PRODUCT_ATTR_VL']), axis = 1)
guarantee['PRODUCT_ATTR_VL'] = guarantee['PRODUCT_ATTR_VL'].astype(int)
guarantee = guarantee.rename(columns = {'PRODUCT_ATTR_VL' : 'guarantee'})

for frame in [brand_country, fab_country, weight, guarantee]:
    frame = frame.rename(columns = {'PRODUCT_ID' : 'SKU'})
    clf_df = clf_df.merge(frame, how = 'left', on = ['SKU'])

    
# Nan filling
# clf_df = clf_df.fillna(0)

print('Frame shape:', clf_df.shape)
clf_df.head()

Frame shape: (98582, 99)


,SKU,Store,Elasticity,Qty,P_value,count,__type__,id,PRICE_REG__index_mass_quantile__q_0.1,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_2",TGT_QTY__ratio_value_number_to_time_series_length,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_15","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_27","PRICE_REG__fft_coefficient__attr_""abs""__coeff_37","PRICE_REG__fft_coefficient__attr_""abs""__coeff_28","PRICE_REG__fft_coefficient__attr_""abs""__coeff_25","PRICE_REG__fft_coefficient__attr_""abs""__coeff_13",PRICE_REG__index_mass_quantile__q_0.2,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_5","PRICE_REG__fft_coefficient__attr_""abs""__coeff_8","PRICE_REG__fft_coefficient__attr_""abs""__coeff_39","PRICE_REG__fft_coefficient__attr_""abs""__coeff_35","PRICE_REG__fft_coefficient__attr_""abs""__coeff_30","PRICE_REG__fft_coefficient__attr_""abs""__coeff_4","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0",PRICE_REG__variation_coefficient,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_11",TGT_QTY__percentage_of_reoccurring_datapoints_to_all_datapoints,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_16","PRICE_REG__linear_trend__attr_""stderr""","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_31","PRICE_REG__fft_coefficient__attr_""abs""__coeff_14","PRICE_REG__fft_coefficient__attr_""abs""__coeff_26","PRICE_REG__fft_coefficient__attr_""abs""__coeff_7","PRICE_REG__fft_aggregated__aggtype_""skew""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","PRICE_REG__fft_coefficient__attr_""abs""__coeff_20","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_34","PRICE_REG__fft_coefficient__attr_""abs""__coeff_33","PRICE_REG__fft_coefficient__attr_""abs""__coeff_38","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","TGT_QTY__fft_coefficient__attr_""abs""__coeff_10",PRICE_REG__spkt_welch_density__coeff_8,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_3","PRICE_REG__fft_coefficient__attr_""imag""__coeff_4","PRICE_REG__fft_coefficient__attr_""abs""__coeff_36","PRICE_REG__fft_coefficient__attr_""abs""__coeff_10","PRICE_REG__fft_coefficient__attr_""abs""__coeff_12","PRICE_REG__fft_coefficient__attr_""abs""__coeff_18","PRICE_REG__fft_aggregated__aggtype_""centroid""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","PRICE_REG__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__lempel_ziv_complexity__bins_100,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_27",TGT_QTY__variance_larger_than_standard_deviation,PRICE_REG__energy_ratio_by_chunks__num_segments_10__segment_focus_1,PRICE_REG__mean_n_absolute_max__number_of_maxima_7,PRICE_REG__spkt_welch_density__coeff_5,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_6","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2",TGT_QTY__index_mass_quantile__q_0.2,"PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""min""",PRICE_REG__sum_of_reoccurring_values,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_9","PRICE_REG__fft_coefficient__attr_""imag""__coeff_13","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__sum_of_reoccurring_values,TGT_QTY__permutation_entropy__dimension_3__tau_1,TGT_QTY__energy_ratio_by_chunks__num_segments_10__segment_focus_0,TGT_QTY__range_count__max_1__min_-1,TGT_QTY__symmetry_looking__r_0.05,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_25","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""","PRICE_REG__fft_coefficient__attr_""

In [43]:
nans = get_nan_ratio(clf_df).head(20)
nans = nans[nans['NaN_ratio_prc'] > 0.5]
clf_df = clf_df.drop(columns = nans['Column'].to_list())
clf_df

,SKU,Store,Elasticity,Qty,P_value,count,__type__,id,PRICE_REG__index_mass_quantile__q_0.1,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_2",TGT_QTY__ratio_value_number_to_time_series_length,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_15","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_27","PRICE_REG__fft_coefficient__attr_""abs""__coeff_28","PRICE_REG__fft_coefficient__attr_""abs""__coeff_25","PRICE_REG__fft_coefficient__attr_""abs""__coeff_13",PRICE_REG__index_mass_quantile__q_0.2,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_5","PRICE_REG__fft_coefficient__attr_""abs""__coeff_8","PRICE_REG__fft_coefficient__attr_""abs""__coeff_30","PRICE_REG__fft_coefficient__attr_""abs""__coeff_4","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0",PRICE_REG__variation_coefficient,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_11",TGT_QTY__percentage_of_reoccurring_datapoints_to_all_datapoints,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_16","PRICE_REG__linear_trend__attr_""stderr""","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_31","PRICE_REG__fft_coefficient__attr_""abs""__coeff_14","PRICE_REG__fft_coefficient__attr_""abs""__coeff_26","PRICE_REG__fft_coefficient__attr_""abs""__coeff_7","PRICE_REG__fft_aggregated__aggtype_""skew""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","PRICE_REG__fft_coefficient__attr_""abs""__coeff_20","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_33","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","TGT_QTY__fft_coefficient__attr_""abs""__coeff_10",PRICE_REG__spkt_welch_density__coeff_8,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_3","PRICE_REG__fft_coefficient__attr_""imag""__coeff_4","PRICE_REG__fft_coefficient__attr_""abs""__coeff_10","PRICE_REG__fft_coefficient__attr_""abs""__coeff_12","PRICE_REG__fft_coefficient__attr_""abs""__coeff_18","PRICE_REG__fft_aggregated__aggtype_""centroid""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","PRICE_REG__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__lempel_ziv_complexity__bins_100,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_27",TGT_QTY__variance_larger_than_standard_deviation,PRICE_REG__energy_ratio_by_chunks__num_segments_10__segment_focus_1,PRICE_REG__mean_n_absolute_max__number_of_maxima_7,PRICE_REG__spkt_welch_density__coeff_5,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_6","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2",TGT_QTY__index_mass_quantile__q_0.2,"PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""min""",PRICE_REG__sum_of_reoccurring_values,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_9","PRICE_REG__fft_coefficient__attr_""imag""__coeff_13","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__sum_of_reoccurring_values,TGT_QTY__permutation_entropy__dimension_3__tau_1,TGT_QTY__energy_ratio_by_chunks__num_segments_10__segment_focus_0,TGT_QTY__range_count__max_1__min_-1,TGT_QTY__symmetry_looking__r_0.05,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_25","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""","PRICE_REG__fft_coefficient__attr_""real""__coeff_14","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0","PRICE_REG__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","PRICE_REG__fft_coefficient__attr_""imag""__coeff_6",TGT_QTY__count_below_mean,TGT_QTY__index_mass_quantile__q_0.3
0,370,223,-0.2740,0.0421,0.1692,190,n

In [44]:
base_df = clf_df[clf_df['__type__'] == 'n0%']
for cr_type in list(set(clf_df['__type__'].unique()) - set(['n0%'])):
    tmp2 = clf_df[clf_df['__type__'] == cr_type][['SKU', 'Store','Elasticity','P_value']]
    tmp2 = tmp2.rename(columns = {'Elasticity' : f'Elasticity_{cr_type}',
                                  'P_value' : f'P_value_{cr_type}'})
    base_df = base_df.merge(tmp2, how = 'left', on = ['SKU', 'Store'])


croped_exct_df = pd.DataFrame()
for base_cr_type in list(set(clf_df['__type__'].unique()) - set(['n0%'])):
    tmp = clf_df[clf_df['__type__'] == base_cr_type]
    tmp2 = double_crops[double_crops['__type1__'] == base_cr_type]

    for cr_type in tmp2['__type2__'].unique():
        tmp3 = tmp2[tmp2['__type2__'] == cr_type][['SKU', 'Store', 'Elasticity', 'P_value']]
        tmp3 = tmp3.rename(columns = {'Elasticity' : f'Elasticity_{cr_type}',
                                      'P_value' : f'P_value_{cr_type}'})
        tmp = tmp.merge(tmp3, how = 'left', on = ['SKU', 'Store'])
    
    croped_exct_df = croped_exct_df.append(tmp)
    
base_df = base_df.append(croped_exct_df[list(base_df.columns)])
clf_df = base_df.copy()

print('Frame shape:', clf_df.shape)
clf_df.head()

Frame shape: (98582, 99)


,SKU,Store,Elasticity,Qty,P_value,count,__type__,id,PRICE_REG__index_mass_quantile__q_0.1,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_2",TGT_QTY__ratio_value_number_to_time_series_length,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_15","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_27","PRICE_REG__fft_coefficient__attr_""abs""__coeff_28","PRICE_REG__fft_coefficient__attr_""abs""__coeff_25","PRICE_REG__fft_coefficient__attr_""abs""__coeff_13",PRICE_REG__index_mass_quantile__q_0.2,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_5","PRICE_REG__fft_coefficient__attr_""abs""__coeff_8","PRICE_REG__fft_coefficient__attr_""abs""__coeff_30","PRICE_REG__fft_coefficient__attr_""abs""__coeff_4","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0",PRICE_REG__variation_coefficient,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_11",TGT_QTY__percentage_of_reoccurring_datapoints_to_all_datapoints,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_16","PRICE_REG__linear_trend__attr_""stderr""","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_31","PRICE_REG__fft_coefficient__attr_""abs""__coeff_14","PRICE_REG__fft_coefficient__attr_""abs""__coeff_26","PRICE_REG__fft_coefficient__attr_""abs""__coeff_7","PRICE_REG__fft_aggregated__aggtype_""skew""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","PRICE_REG__fft_coefficient__attr_""abs""__coeff_20","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","PRICE_REG__fft_coefficient__attr_""abs""__coeff_33","PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","TGT_QTY__fft_coefficient__attr_""abs""__coeff_10",PRICE_REG__spkt_welch_density__coeff_8,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_3","PRICE_REG__fft_coefficient__attr_""imag""__coeff_4","PRICE_REG__fft_coefficient__attr_""abs""__coeff_10","PRICE_REG__fft_coefficient__attr_""abs""__coeff_12","PRICE_REG__fft_coefficient__attr_""abs""__coeff_18","PRICE_REG__fft_aggregated__aggtype_""centroid""","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","PRICE_REG__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__lempel_ziv_complexity__bins_100,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_27",TGT_QTY__variance_larger_than_standard_deviation,PRICE_REG__energy_ratio_by_chunks__num_segments_10__segment_focus_1,PRICE_REG__mean_n_absolute_max__number_of_maxima_7,PRICE_REG__spkt_welch_density__coeff_5,"PRICE_REG__fft_coefficient__attr_""abs""__coeff_6","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2",TGT_QTY__index_mass_quantile__q_0.2,"PRICE_REG__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""min""",PRICE_REG__sum_of_reoccurring_values,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_9","PRICE_REG__fft_coefficient__attr_""imag""__coeff_13","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",TGT_QTY__sum_of_reoccurring_values,TGT_QTY__permutation_entropy__dimension_3__tau_1,TGT_QTY__energy_ratio_by_chunks__num_segments_10__segment_focus_0,TGT_QTY__range_count__max_1__min_-1,TGT_QTY__symmetry_looking__r_0.05,"PRICE_REG__fft_coefficient__attr_""imag""__coeff_25","PRICE_REG__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""","PRICE_REG__fft_coefficient__attr_""real""__coeff_14","PRICE_REG__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0","PRICE_REG__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","PRICE_REG__fft_coefficient__attr_""imag""__coeff_6",TGT_QTY__count_below_mean,TGT_QTY__index_mass_quantile__q_0.3,Elasticity_f40%,P_value_f40%,Elastici

In [45]:
# clf_df.to_csv('clf_df.csv', index = False)

In [46]:
clf_df.to_csv('clf_df.csv', index = False)

----
----
----